# Log

* start time oct 24 - 21:10 - 01:06
* read dqn paper nov 19 - 22:23 - 22:55
* add fog nov 25 - 23:15 - 23:33
* more on moves nov 26 - 8:00 - 9:00 | 9:49 - 12:21
* reading pp0 paper nov 26 - 12:30 - 12:57
* conceptualizing project search space/ reward systems nov 26 - 13:50 - 15:32
* more conceptualizing nov 26 - 15:57 - 17:24
* preparing for PPO using simple rewards 17:30 - 18:54 minus 30 min distracted
* connect PPO with game nov 26 - 19:33 - 20:44
* still connecting nov 26 - 21:13 - 22:12
* finally starting to run nov 26 - 22:12 -

#Game

In [13]:
#python3


import numpy as np


class Map_thing():
  id = 0
  def __init__(self, t_char, mohs):
    self.t_char = t_char
    self.mohs = mohs
    self.id = Map_thing.id
    self.row = None
    self.col = None
    Map_thing.id += 1

  def print_self(self):
    return self.t_char

  def get_mohs(self):
    return self.mohs

  def __repr__(self):
    return self.t_char

  def __eq__(self, other):
    return other != None and self.t_char == other.t_char and self.id == other.id

  def set_location(self, row, col):
    self.row = row
    self.col = col

class Controller():
  def __init__(self, move_func=None, responder=None, creature=None, looker=None):
    self.move_func = move_func if move_func is not None else self._random
    self.responder = responder if responder is not None else print
    self.looker = looker if looker is not None else print
    self.creature = creature

  def _random(self):
    x = np.random.randint(0, 4)
    if x == 0:
      return 'north'
    elif x == 1:
      return 'south'
    elif x == 2:
      return 'east'
    elif x == 3:
      return 'west'
    return x

  def look(self, look_info):
    self.looker(look_info)

  def canMove(self):
    if self.creature != None:
      return self.creature.canMove()
    return False
  
  def getMove(self):
    return self.move_func()
  
  def sendResponse(self, message):
    self.responder(message)

  def setCreature(self, creature):
    self.creature = creature

  def getCreature(self):
    return self.creature

class SimpleController(Controller):
  def __init__(self, creature=None, prey=None, responder=None, looker=None):
    super().__init__(creature=creature, move_func=self._getMove, responder=responder, looker=looker)
    self.prey = prey
    if (self.creature == None or self.prey == None):
      self.move_func = super()._random
    self.last = ['', '']

  def _try_row(self, row_diff):
    result = ''
    if row_diff > 0:
        result = 'north'
    elif row_diff < 0:
        result = 'south'
    return result

  def _try_col(self, col_diff):
    result = ''
    if col_diff > 0:
        result = 'west'
    elif col_diff < 0:
        result = 'east'
    return result

  def sendResponse(self, message):
    self.last[1] = message
    super().sendResponse(message)

  def _getMove(self, fudge=0):
    row_diff = (self.creature.row - self.prey.row) + fudge
    col_diff = self.creature.col - self.prey.col

    result = ''
    # print(self.creature.id, row_diff, col_diff)
    if abs(row_diff) > abs(col_diff):
      result = self._try_row(row_diff)
      if result == '':
        result = self._try_col(col_diff)
    elif abs(row_diff) < abs(col_diff):
      result = self._try_col(col_diff)
      if result == '':
        result = self._try_row(row_diff)
    else:
      result = self._getMove(np.random.rand() - .5) # make even chance

    if result == '':
      result = self._random()      

    # super().sendResponse(str(result))
    self.last = [result, '']
    return result

class ConsoleController(Controller):
  def __init__(self, creature=None, responder=None):
    super().__init__(creature=creature, move_func=self._getMove, responder=responder)
  
  def _getMove(self):
    result = input("Your move >")
    return result

class Creature(Map_thing):
  def __init__(self, t_char, mohs, num_moves=1, wait_moves=0, drop_thing=None):
    super().__init__(t_char, mohs)
    self.num_moves = num_moves
    self.max_wait = wait_moves
    self.wait_count = self.max_wait
    self.drop_thing = drop_thing

  def canMove(self):
    if self.wait_count <= 0:
      self.wait_count = self.max_wait
      return True
    self.wait_count -= 1
    return False

  def getNumMoves(self):
    return self.num_moves

  def getMove(self):
    return self.controller.getMove()

  def sendResponse(self, message):
    self.controller.sendResponse(message)

  def get_drop(self):
    return self.drop_thing

class Map:
  def __init__(self, width, height, density=20):
    self.width = width
    self.height = height
    self.density = density

    self._wall = Map_thing("█",10)
    self._space = Map_thing(' ',0)
    self._fog = Map_thing('*', 10)

    self._map = []
    self.re_init_map(self.density)

  def re_init_map(self, density=None):
    den = density if density is not None else self.density
    self._map = []
    for row in range(self.height):
      the_row = []
      if row == 0 or row == self.height - 1:
        the_row = [[self._wall] for _i in range(self.width)]
      else:
        the_row = [[self._wall]]
        the_row.extend([[self._wall] if np.random.randint(100) <= den else [self._space] for i in range(self.height - 2)])
        the_row.append([self._wall])
      self._map.append(the_row)

  def place(self, map_thing, to_row, to_col):
    worked = True
    if to_row <= 0 or to_row >= self.height:
      worked = False
    if to_col <= 0 or to_col >= self.width:
      worked = False
    if map_thing.mohs <= self._map[to_row][to_col][-1].mohs:
      worked = False
    if not worked:
      return 'Bad Move'
    self._map[to_row][to_col].append(map_thing)
    map_thing.set_location(to_row, to_col)
    return 'Moved'

  def rand_place(self, map_thing, tries=10):
    result = ''
    for _x in range(tries):
      result = self.place(map_thing, np.random.randint(1, self.height -1), np.random.randint(1, self.width - 1))
      if result == 'Moved':
        return result
    return result


  def move(self, creature, from_row, from_col, move):
    to_row = from_row
    to_col = from_col

    if move == 'north' or move == 0:
      to_row -= 1
    elif move == 'south' or move == 1:
      to_row += 1
    elif move == 'west' or move == 2:
      to_col -= 1
    elif move == 'east' or move == 3:
      to_col += 1
    else:
      return 'Unknown Move'
    
    place_result = self.place(creature, to_row, to_col)
    if place_result == 'Moved':
      self._map[from_row][from_col].remove(creature)
      if creature.get_drop() != None:
        self._map[from_row][from_col].append(creature.get_drop())
    return place_result

  def move_creature(self, map_creature, move):
    if map_creature != None and map_creature.row != None and map_creature.col != None:
      return self.move(map_creature, map_creature.row, map_creature.col, move)
    return 'Location Unknown'

  def print(self, map_thing=None, flashlight=5):
    map_str = ''
    for r in range(self.height):
      if (map_thing != None and map_thing.row != None and map_thing.col != None) and (r < map_thing.row - flashlight or r > map_thing.row + flashlight):
        map_str += self._fog.t_char * self.width
      else:
        for c in range(self.width):
          if map_thing != None and (c < map_thing.col - flashlight or c > map_thing.col + flashlight):
            map_str += self._fog.t_char
          else:
            map_str += self._map[r][c][-1].t_char
      map_str += '\n'
    return map_str

  def __len__(self):
    return self.width * self.height
  
  def __repr__(self):
    return self.print()

class Game:
  def __init__(self, size=(10, 10), level=0, player_controller=None, smart_controller=None):
    self.level = level
    self.map = Map(*size, density=20)
    self.resets = 4
    self.goal = Map_thing('@', 1)
    self.player = Creature('#', 2)
    self.player_controller = SimpleController(self.player, self.goal) if player_controller == None else player_controller
    # self.player.setController(self.player_controller)
    self.player_controller.setCreature(self.player)
    self.controllers = [self.player_controller]
    self.vampires = []
    self.other_creatures = []
    self.init_map()


  def init_map(self):
    self.turns_taken = 0
    self.map.re_init_map()

    vamp_len = len(self.vampires)

    def no_print(message):
      pass
    
    while len(self.vampires) < self.level:
      vamp = Creature('V', 5, num_moves=2)
      control = SimpleController(vamp, self.player, looker=no_print, responder=no_print)
      self.controllers.append(control)
      self.vampires.append(vamp)

    wrappings = Map_thing('W', 7)
    for level in range(vamp_len, self.level):
      if self.level != 0 and level != 0:
        if level % 5 == 0:
          mummy = Creature('M', 8, drop_thing=wrappings)
          control = SimpleController(mummy, self.player, looker=no_print, responder=no_print)
          self.controllers.append(control)
          self.other_creatures.append(mummy)
        if level % 10 == 0:
          ghost = Creature('G', 11, wait_moves=5)
          control = SimpleController(ghost, self.player, looker=no_print, responder=no_print)
          self.controllers.append(control)
          self.other_creatures.append(ghost)

    for v in self.vampires:
      self.map.rand_place(v)
    for c in self.other_creatures:
      self.map.rand_place(c)  
    
    self.map.rand_place(self.player)
    self.map.rand_place(self.goal)

  def check_win(self):
    win = self.player.row == self.goal.row and self.player.col == self.goal.col
    if win:
      self.level += 1
      self.turns_taken = 0
    return win

  def check_loss(self):
    for c in self.vampires:
      if self.player.row == c.row and self.player.col == c.col:
        return True
    for other in self.other_creatures:
      if self.player.row == other.row and self.player.col == other.col:
        return True
    return False

  def broadcastMessage(self, message):
    for controller in self.controllers:
      controller.sendResponse(message)

  def run(self, turns=None, max_level=50):
    turns = len(self.map) * 2 if turns == None else turns
    player_wins = False
    while self.turns_taken < turns and self.level <= max_level:
      self.turns_taken += 1
      for controller in self.controllers:
        map_info = self.map.print(map_thing=controller.getCreature(), flashlight=len(self.map))
        controller.look(map_info)
        if controller.canMove():
          move = controller.getMove()
          if (move == 'reset' or move == 4) and self.resets > 0:
            self.resets -= 1
            # controller.sendResponse(str(self.resets))
            self.broadcastMessage('Reset ' + str(self.level))
            self.init_map()
            break
          if (move == 'quit' or move == 5):
            self.turns_taken = turns + 1
            self.broadcastMessage('Quitting! ' + str(self.level))
            return 'Quitting'
          move_result = self.map.move_creature(controller.getCreature(), move)
          player_wins = self.check_win()
          if player_wins:
            self.broadcastMessage('Player leveled! ' + str(self.level) + ' ' + self.goal.t_char)
            self.init_map()
          if self.check_loss():
            self.broadcastMessage('Player lost! ' + str(self.level) + ' ' + controller.getCreature().t_char)
            return 'Loss'
        else:
          controller.sendResponse("Can't move " + str(self.level))

    if self.turns_taken >= turns:
      self.broadcastMessage('Too long! ' + str(self.level))
      return 'Too Long'
    self.broadcastMessage('Max level reached! ' + str(self.level))
    return 'Max level'





if __name__ == "__main__":

  # g = Game(level=11, size=(20, 20), player_controller=ConsoleController())
  g = Game(level=1, size=(20, 20), player_controller=None)

  g.run(max_level=3, turns=10)
  print(g.map)

  



████████████████████
██ ███   █  ██     █
█ █ █       █  █   █
█   ███#         █ █
█        █         █
█         ██ █ █   █
█                  █
██         █       █
█       █    █    ██
█    █         █████
█               ██ █
█  █ █ █ █  @█     █
█  █      █        █
██   █       V █  ██
█ █         █      █
█ █    █ █         █
█   █        █    ██
█     ███    █ █   █
██      █          █
████████████████████

████████████████████
██ ███   █  ██     █
█ █ █       █  █   █
█   ███          █ █
█      # █         █
█         ██ █ █   █
█                  █
██         █       █
█       █    █    ██
█    █         █████
█               ██ █
█  █ █ █ █  @█     █
█  █      █  V     █
██   █         █  ██
█ █         █      █
█ █    █ █         █
█   █        █    ██
█     ███    █ █   █
██      █          █
████████████████████

████████████████████
██ ███   █  ██     █
█ █ █       █  █   █
█   ███          █ █
█        █         █
█      #  ██ █ █   █
█                  █
██         

#Planning

Okay, so the game works in general now.  I need to find a couple parameters to search over.  

In my proposal, I proposed iterating over different:


1.   Map sizes
2.   Map densities
3.   Level progressions
4.   Flashlight y (distance?) /n


As well, there is the question of when to introduce the Vampire Lord.  The possibilities include:
*    Never introduce vampire lord
      *   Train player with no vampire lord

*    Independant training:
      1.   Train player with no vampire lord
      2.   Train vampire lord with dumb player
      3.   Train them together

*    In-course training:
      *   Train player
      *   Introduce vampire lord at level X
    

I should have enough responses built in to build reward schemes.

Reward for speed completion?




#Events


*   Player completed a level
*   Player caught (specific monster?) - terminates
*   too long (bad for everyone) - terminates
*   quit (very bad for player) - terminates
*   Player reset map


The following reward schemes are being developed in paralell and iterated upon.

## Simple:

The simplest reward scheme for the player would be:

*   +1 for every level progressed
*   -1 for caught by monster
*   -1 for too long
*   -L with L being the level achieved on quit
*   +0 otherwise

The simplest reward scheme for the Vampire Lord would be:

*   +1 for the player being caught
*   -1 for the player progressing a level
*   -1 for too long
*   +0 otherwise





### Discussion:
This will allow the player an increasing reward for progressing through levels and penalize the Vampire Lord for letting that happen.  It will also let the player quit, but then it will lose all of its points.

With the simulation:
```
score = 0
for l in range(1, 8):
    print('level', l, 'for complete', score + 1, 'caught', score-1, 'too long', score -1, 'on quit', score - l)
    score += 1
```
the results:
```
level 1 for complete 1 caught -1 too long -1 on quit -1
level 2 for complete 2 caught 0 too long 0 on quit -1
level 3 for complete 3 caught 1 too long 1 on quit -1
level 4 for complete 4 caught 2 too long 2 on quit -1
level 5 for complete 5 caught 3 too long 3 on quit -1
level 6 for complete 6 caught 4 too long 4 on quit -1
level 7 for complete 7 caught 5 too long 5 on quit -1
```
I like how for the first two levels, the only way to get any score is to progress.  And if the player is caught, at least it will have some reward for what progress it has made.  However, I'm not sure if I like the balance between the quitting and the too long penalty.

Perhaps I may change the penalty for quitting to `-2`.  That way I can still assert that I don't like it to quit, but it would still get some reward.



## Complex:

A more complex reward scheme for player:

*   +L with L being the level completed
*   -T/L with T being the sum of all levels cleared when caught by a monster
*   -T/L when too long
*   -1 for each reset
*   -T on quit
*   +0 otherwise

A more complex reward scheme for the Vampire Lord:

*   +1 for if the player is caught
*   +2 for if the player is caught by the Vampire Lord
*   -1 for if the player progresses a level
*   -2 for if goes too long
*   +0 otherwise


### Discussion:
The player scheme is way to complex.  For a simulation of:
```
score = 0
for l in range(1, 8):
    summer = sum([x+1 for x in range(l)])
    avg = summer / l
    print('level', l, 'for complete', score + l, 'caught', score-avg, 'too long', score -avg, 'on quit', score - summer)
    score += l
```
The result of:
```
level 1 for complete 1 caught -1.0 too long -1.0 on quit -1
level 2 for complete 3 caught -0.5 too long -0.5 on quit -2
level 3 for complete 6 caught 1.0 too long 1.0 on quit -3
level 4 for complete 10 caught 3.5 too long 3.5 on quit -4
level 5 for complete 15 caught 7.0 too long 7.0 on quit -5
level 6 for complete 21 caught 11.5 too long 11.5 on quit -6
level 7 for complete 28 caught 17.0 too long 17.0 on quit -7
```
This has the same benefit on the first two levels as the simple reward scheme.  Notice the MUCH bigger penalty for quitting than for being caught.  

## Middle Ground

The reward scheme for the player would be:

*   +2 for every level completed
*   -1 for caught by monster
*   -2 for too long  ?? debate??
*   -L with L being the current level on quit ?? debate??
*   +0 otherwise

The reward scheme for the Vampire Lord would be:

*   +1 for if the player is caught
*   +2 for if the player is caught by the Vampire Lord
*   -1 for if the player progresses a level
*   -2 for if goes too long
*   +0 otherwise


### Debate
For the player I need to prioritize penalties between being caught, going for too long, or quitting.  Alternatively, I could just make it so that the playerAI does not have the option of quitting.

The top prioritiy for the player is to clear levels, and progress to the next one.  In order to do that, it has to avoid getting caught.  Stalling, or going for too long, does not help anyone play well.  For normal play, a human may quit while they are ahead.

Using the simulation:
```
score = 0
for l in range(1, 8):
    print('level', l, 'for complete', score + 2, 'caught', score-1, 'too long', score -2, 'on quit', score - l)
    score += 2
```
The result of:
```
level 1 for complete 2 caught -1 too long -2 on quit -1
level 2 for complete 4 caught 1 too long 0 on quit 0
level 3 for complete 6 caught 3 too long 2 on quit 1
level 4 for complete 8 caught 5 too long 4 on quit 2
level 5 for complete 10 caught 7 too long 6 on quit 3
level 6 for complete 12 caught 9 too long 8 on quit 4
level 7 for complete 14 caught 11 too long 10 on quit 5
```
Shows that the player will get more of a reward for any level if it gets caught compared to if it quits.

I think I may want to encourage this, because this will *let* the player allow the Vampire Lord to learn, rather than quitting too soon and 'starving' the Vampire Lord of material.

#Policy Translator

In [14]:
def translate_player_policy(result_str, which='simple'):
  results = result_str.split()
  results[1] = int(results[1])
  if which is 'simple':
    policy = {'Quitting!': -1 * results[1], 'Player leveled!': 1, 'Player lost!': -1, 'Too long!': -1}
    return policy[results[0]] if results[0] in policy.keys else 0
  if which is 'complex':
    Total = -1 * sum([x for x in range(results[1])])
    T_over_L = Total/results[1]
    policy = {'Quitting!': Total, 'Player leveled!': int(results[1]), 'Player lost!': T_over_L, 'Too long!': T_over_L, 'Reset': -1}
    return policy[results[0]] if results[0] in policy.keys else 0
  if which is 'middle':
    policy = {'Quitting!': results[1], 'Player leveled!': +2, 'Player lost!': -1, 'Too long!': -2}
    return policy[results[0]] if results[0] in policy.keys else 0
  # default simple
  policy = {'Quitting!': -1 * results[1], 'Player leveled!': 1, 'Player lost!': -1, 'Too long!': -1}
  return policy[results[0]] if results[0] in policy.keys else 0



In [15]:
import numpy as np
import gc

def map_to_numpy(map_str):
  one_dim = map_str.split('\n')[:-1]
  two_dim = [[ord(y) for y in x] for x in one_dim]
  two_dim = np.array(two_dim)
  # print(two_dim)
  return two_dim

mmm = Map(10, 10)

map_to_numpy(mmm.print())

array([[9608, 9608, 9608, 9608, 9608, 9608, 9608, 9608, 9608, 9608],
       [9608,   32,   32,   32,   32,   32,   32,   32,   32, 9608],
       [9608, 9608,   32,   32,   32,   32,   32,   32,   32, 9608],
       [9608,   32,   32,   32,   32,   32,   32,   32,   32, 9608],
       [9608, 9608, 9608,   32,   32,   32,   32,   32,   32, 9608],
       [9608, 9608,   32, 9608, 9608, 9608,   32,   32, 9608, 9608],
       [9608,   32, 9608, 9608,   32,   32,   32,   32,   32, 9608],
       [9608,   32,   32,   32,   32,   32, 9608,   32,   32, 9608],
       [9608,   32, 9608, 9608, 9608, 9608, 9608, 9608,   32, 9608],
       [9608, 9608, 9608, 9608, 9608, 9608, 9608, 9608, 9608, 9608]])

I wonder, is it going to be bad that I am using such estoric characters to represent the map...or is there an encoder that I can use...?

#Networks


In [28]:
import gym
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


# Dataset that wraps memory for a dataloader
class RLDataset(Dataset):
  def __init__(self, data):
    super().__init__()
    self.data = []
    for d in data:
      self.data.append(d)
  
  def __getitem__(self, index):
    return self.data[index]
 
  def __len__(self):
    return len(self.data)

# Policy Network from lab9
class PolicyNetwork(nn.Module):
  def __init__(self, state_size, action_size):
    super().__init__()
    hidden_size = 8
    
    self.net = nn.Sequential(nn.Linear(state_size, hidden_size),
                             nn.ReLU(),
                             nn.Linear(hidden_size, hidden_size),
                             nn.ReLU(),
                             nn.Linear(hidden_size, hidden_size),
                             nn.ReLU(),
                             nn.Linear(hidden_size, action_size),
                             nn.Softmax(dim=1))
  
  def forward(self, x):
    """Get policy from state

      Args:
          state (tensor): current state, size (batch x state_size)

      Returns:
          action_dist (tensor): probability distribution over actions (batch x action_size)
    """
    print(x, x.size())
    print(self.net[0].weight.size())
    return self.net(x)


# Value Network from lab9
class ValueNetwork(nn.Module):
  def __init__(self, state_size):
    super().__init__()
    hidden_size = 8
  
    self.net = nn.Sequential(nn.Linear(state_size, hidden_size),
                             nn.ReLU(),
                             nn.Linear(hidden_size, hidden_size),
                             nn.ReLU(),
                             nn.Linear(hidden_size, hidden_size),
                             nn.ReLU(),
                             nn.Linear(hidden_size, 1))
    
  def forward(self, x):
    """Estimate value given state

      Args:
          state (tensor): current state, size (batch x state_size)

      Returns:
          value (tensor): estimated value, size (batch)
    """
    return self.net(x)

#Helper Functions

In [17]:
def calculate_return(memory, rollout, gamma):
  """Return memory with calculated return in experience tuple

    Args:
        memory (list): (state, action, action_dist, return) tuples
        rollout (list): (state, action, action_dist, reward) tuples from last rollout
        gamma (float): discount factor

    Returns:
        list: memory updated with (state, action, action_dist, return) tuples from rollout
  """
  reversed_return = []
  for item in reversed(rollout):
    the_return = 0
    if len(reversed_return) > 0:
      the_return = item[-1] + gamma*reversed_return[-1][-1]
    else:
      the_return = item[-1]
    reversed_return.append(item[:-1] + (the_return,))

  reversed_return.reverse()
  return memory + reversed_return


def get_action_ppo(network, state):
  """Sample action from the distribution obtained from the policy network

    Args:
        network (PolicyNetwork): Policy Network
        state (np-array): current state, size (state_size)

    Returns:
        int: action sampled from output distribution of policy network
        array: output distribution of policy network
  """
  with torch.no_grad():
    torch_state = torch.as_tensor(state, device='cuda').float().unsqueeze(0)
    distribution = network(torch_state).detach()
    action = torch.multinomial(distribution, 1).item()
    return action, distribution
  

def learn_ppo(optim, policy, value, memory_dataloader, epsilon, policy_epochs):
  """Implement PPO policy and value network updates. Iterate over your entire 
     memory the number of times indicated by policy_epochs.    

    Args:
        optim (Adam): value and policy optimizer
        policy (PolicyNetwork): Policy Network
        value (ValueNetwork): Value Network
        memory_dataloader (DataLoader): dataloader with (state, action, action_dist, return, discounted_sum_rew) tensors
        epsilon (float): trust region
        policy_epochs (int): number of times to iterate over all memory
  """
  optim.zero_grad()

  for count in range(policy_epochs):
    optim.zero_grad()
    for batch in memory_dataloader:
      state, action, action_dist, theReturn = batch

      torch_state = torch.as_tensor(state, device='cuda').float()
      actions = torch.as_tensor(action.view(-1, 1), device='cuda')
      torch_return = torch.as_tensor(theReturn, device='cuda')

      advantages = torch_return - value(torch_state)
      value_loss = (advantages**2).mean()
      advantages = advantages.detach()

      pi = torch.gather(action_dist.squeeze(1).cuda(), 1, actions).squeeze()
      piprime = torch.gather(policy(torch_state), 1, actions).squeeze()
      ratio = piprime / pi
      policy_loss = -1 * torch.mean(torch.min(ratio * advantages, torch.clamp(ratio, 1-epsilon, 1+epsilon) * advantages))

      total_loss = value_loss + policy_loss
      
      total_loss.backward()
      optim.step()


#NetController

In [22]:
class NetController(Controller):
  
  def __init__(self, net):
    super().__init__(move_func=self._getMove, responder=self._responder, looker=self._looker)
    self.net = net
    self.memory = []
    self.rewards = []
    self.cumulative_reward = 0
    self.rollout = []
    self.cur_state = None
    # self.move_func = self._getMove
    # self.looker = self._looker
    # self.responder = self._responder

  def epoch_reset(self):
    self.memory = []
    self.rewards = []
    self.rollout = []
    self.cumulative_reward = 0

  def flush_episode(self, gamma):
    self.memory = calculate_return(self.memory, self.rollout, gamma)
    self.rewards.append(self.cumulative_reward)
    self.cumulative_reward = 0

  def _looker(self, map_str):
    self.cur_state = map_to_numpy(map_str)

  def _getMove(self):
    print(self.cur_state)
    self.action, self.action_dist = get_action_ppo(self.net, self.cur_state)
    return self.action

  def _responder(self, message):
    reward = translate_player_policy(message)
    self.cumulative_reward += reward
    rollout.append((self.cur_state, self.action, self.action_dist, reward))

#Main PPO Algorithm

In [29]:
from itertools import chain
import matplotlib.pyplot as plt
from tqdm import tqdm


def ppo_main():
  # Hyper parameters
  lr = 1e-3
  epochs = 20
  env_samples = 100
  gamma = 0.9
  batch_size = 256
  epsilon = 0.2
  policy_epochs = 5

  # Init environment varables
  map_width = 40
  map_height = 40
  player_actions = 6

  # Init networks
  player_policy_network = PolicyNetwork(map_width * map_height, player_actions).cuda()
  player_value_network = ValueNetwork(map_width * map_height).cuda()

  # init controllers
  player_controller = NetController(player_policy_network)

  # Init optimizer
  optim = torch.optim.Adam(chain(player_policy_network.parameters(), player_value_network.parameters()), lr=lr)

  # Start main loop
  results_ppo = []
  loop = tqdm(total=epochs, position=0, leave=False)
  for epoch in range(epochs):
    
    # memory = []  # Reset memory every epoch
    # rewards = []  # Calculate average episodic reward per epoch
    player_controller.epoch_reset()

    # Begin experience loop
    for episode in range(env_samples):

      game = Game(size=(map_width, map_height), player_controller=player_controller)
      game.run(max_level=16)
      player_controller.flush_episode(gamma)
      
      
    # Train
    dataset = RLDataset(player_controller.memory)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    learn_ppo(optim, player_policy_network, player_value_network, loader, epsilon, policy_epochs)
    
    # Print results
    results_ppo.extend(player_controller.rewards)  # Store rewards for this epoch
    loop.update(1)
    loop.set_description("Epochs: {} Reward: {}".format(epoch, results_ppo[-1]))

  return results_ppo

results_ppo = ppo_main()

  0%|          | 0/20 [00:00<?, ?it/s]

[[9608 9608 9608 ... 9608 9608 9608]
 [9608   32 9608 ...   32   32 9608]
 [9608   32 9608 ...   32   32 9608]
 ...
 [9608   32 9608 ...   32 9608 9608]
 [9608   32   32 ...   32   32 9608]
 [9608 9608 9608 ... 9608 9608 9608]]
tensor([[[9608., 9608., 9608.,  ..., 9608., 9608., 9608.],
         [9608.,   32., 9608.,  ...,   32.,   32., 9608.],
         [9608.,   32., 9608.,  ...,   32.,   32., 9608.],
         ...,
         [9608.,   32., 9608.,  ...,   32., 9608., 9608.],
         [9608.,   32.,   32.,  ...,   32.,   32., 9608.],
         [9608., 9608., 9608.,  ..., 9608., 9608., 9608.]]], device='cuda:0') torch.Size([1, 40, 40])
torch.Size([8, 1600])


RuntimeError: ignored

In [ ]:
plt.plot(results_ppo)
plt.show()